1.  冷启动场景: 一个纯新用户，如果以某种方式关联到旧用户，我们使用此评分卡；
    1. --训练过程中，只有 新人吃旧人 才能被放入损失， 旧人吃新人 无损失；
         补充； sup 样本也可以与新人匹配， 被新人吃；
    2. 具体呢？ 在冷启动场景下（新用户，非孤点）
    3. --给定cutoff 与 col4全图， 全图中，我有一堆负样本  打一枪就跑， 同时我有一堆正样本， 他们现在是老用户，曾经新过， 我拥有全量用户 profile feat,  不仅有正负样本，还有sup 样本，比如1. 完全拒绝用户； 2. 蜻蜓点水用户；sup 样本仅仅提供关联，不提供损失； 所谓样本 即 target nodes,  它指向 source nodes;  那么根据上面5， target nodes 必须是新人， source nodes 必须是旧人， 新旧用户如何区分？ 根据loan 区分，基于 冷启动场景， user 与 loan 可以同一；即表明上是 userid edge.  实际上要加一层判定，判定loan 先后顺序；区分出source node row & target node row; 针对target node 三分train, valid, test;  
    4. 整体设计上还有遗漏？
    5. --baseline 设计，only profile feats;

1. 首先col4 区分出source node row & target node row; 针对target node 三分train, valid, test;  
2. 然后对target node， 拿到user_profile
3. lgb baseline
4. 图算法

In [3]:
edge_origin_root = '/data-0/qibo/Gdata/origin/'

1. 1 从原始边col_4.csv拿到, 区分source,  target， sup, （新旧无关）

In [1]:
import pandas as pd
import collections
import pickle
from scipy.sparse import csr_matrix
from scipy import sparse
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

In [5]:
col_4_edge = pd.read_csv(edge_origin_root+'col_4.csv', header=None)
col_4_edge = col_4_edge.iloc[:,:2].values
unique_user = list(set(flatten(col_4_edge)))

1.  2 为了区分新旧， 首先要有时间；用户第一笔loan 时间;
       从loan 表中，拿到第一笔fundime 不是non;
       为了区分无关， 是否蜻蜓点水和无情拒绝；要有num_fund==0;
       从虎图结果可以解决; 

In [6]:
root = '/data-0/qibo/Gdata/col_4/raw'
with open(root + '/col_name.pkl', 'rb') as f:
    col_names = pickle.load(f)
x = sparse.load_npz(root + "/feat.npz")
df = pd.DataFrame(x.toarray(), columns=col_names)
df.index =  unique_user

In [28]:
loanlist = pd.read_csv('/data-0/qibo/Gdata/RUN_LIST/loanlist_180608.csv', header=None)


In [37]:
loanlis = loanlist[[0,2]]
userid2fundtime = loanlis.set_index(keys=0).to_dict()[2]

In [46]:
col_4_edge[0]

array([10156519,  9146882])

In [50]:
from datetime import datetime as dt
a = dt.strptime(userid2fundtime[10156519], '%Y-%m-%d %H:%M:%S')
b = dt.strptime(userid2fundtime[9146882], '%Y-%m-%d %H:%M:%S')

In [55]:
col_4_edge[:5]

array([[10156519,  9146882],
       [ 1076207,   813787],
       [ 1076625,   805878],
       [13359919, 13357959],
       [ 5052294,  4861031]])

? 为什么 col_4_edge 中有一些user 在 runlist_180608中不存在？
因为 fundtime is not null； 因此，sup user bu cu b

In [ ]:
讲道理，

In [56]:
from datetime import datetime as dt
print(col_4_edge[:5])
print('=========================')
for i in col_4_edge[:5]:
    t1 = userid2fundtime[i[0]]
    t2 = userid2fundtime[i[1]]
    t1 = dt.strptime(t1, '%Y-%m-%d %H:%M:%S')
    t2 = dt.strptime(t2, '%Y-%m-%d %H:%M:%S')
    if t1>t2:
        print('----------------------')
        print(i)
        i[0],i[1] = i[1], i[0]
        print(i)
print('=========================')
print(col_4_edge[:5])

----------------------
[10156519  9146882]
[ 9146882 10156519]
----------------------
[1076625  805878]
[ 805878 1076625]
----------------------
[13359919 13357959]
[13357959 13359919]


KeyError: 4861031

In [ ]:
def edges2target_source_nodes(col_edge, userid2fundtime):
    
    return target_nodes, source_nodes

In [7]:
sup_users = df[df.fund==0].index.tolist()
fund_users = list(set(unique_user)-set(sup_users))

1. 3 先针对这个边关系做个分析； 

    unfunded用户比例0.34;

    除去unfunded用户后， 打一枪就跑的占24%;

    最终还是黑化的占0.91;
    
    除去打一枪就跑，除去sup用户， 剩下的由白转黑用户比例：0.66; 
    
    如果直接做成规则，那么无疑是损失了 ‘机会’---- 要看在由白转黑的过程中，这col4关系中的66%的用户到底有没有赚钱，赚多少，赚的不多就直接拒绝; 
    
    做一下这个分析; 

In [62]:
print(final_bad.index.to_list())

[7340032, 10485766, 8388615, 9437193, 4194314, 10485796, 5242927, 7340081, 10485826, 9437254, 12582993, 11534430, 3145828, 4194406, 3145836, 9437296, 5242993, 2097268, 11534452, 8388735, 1048705, 5243017, 4194442, 11534479, 7340176, 4194460, 11534497, 7340194, 7340195, 8388774, 4194478, 11534530, 1048778, 4194508, 5243087, 2097365, 9437405, 11534558, 7340257, 14680291, 6291686, 3145962, 6291691, 6291690, 4194540, 9437422, 6291699, 7340275, 13631738, 11534587, 1048831, 2097414, 2097415, 3145992, 1048842, 10486031, 2097445, 7340335, 9437492, 13631797, 4194617, 1048891, 11534649, 3146050, 10486085, 12583242, 7340364, 9437524, 9437526, 9437528, 1048928, 11534694, 10486129, 9437556, 1048949, 12583287, 6291832, 1048954, 9437564, 5243272, 8389002, 9437583, 6291856, 13631890, 6291872, 6291875, 14680494, 8389046, 5243329, 4194758, 1049036, 1049039, 1049040, 4194768, 3146198, 9437655, 7340504, 11534810, 4194779, 1049068, 11534828, 2097647, 11534831, 8389113, 4194809, 1049082, 9437695, 5243389, 1

In [8]:
df_fund = df[df.index.isin(fund_users)] # 钱给你；
one_shot_bad = df_fund[(df_fund.fund==1)&(df_fund.paidoff==0)] # 老子打一枪就跑；
final_bad = df_fund[df_fund.fund!=df_fund.paidoff] # 最终还是没撑住，黑化了；
few_shot_bad = final_bad[~final_bad.index.isin(one_shot_bad.index)]
print('{}\n{}\n{}\n{}'.format(1 - df_fund.shape[0]/df.shape[0], one_shot_bad.shape[0]/len(fund_users), final_bad.shape[0]/len(fund_users), few_shot_bad.shape[0]/len(fund_users)))


0.34602559240374753
0.2474483659232276
0.9106627375511186
0.663214371627891


In [58]:
fund_time

,id,fundtime
userid,,
5004,20,2014-12-10 18:49:30
5011,8099,2015-01-13 12:21:43
5012,17443,2015-01-30 22:10:08
5018,60,2014-12-11 14:24:18
5019,97,2014-12-12 12:18:24
5021,108,2014-12-13 14:32:30
5024,208,2014-12-13 18:13:18
5037,10408,2015-01-20 14:09:08
5059,148,2014-12-13 17:01:55


In [5]:
col_dir = '/data-0/qibo/Gdata/RUN_LIST/loanlist_1806_1807.csv'

one_month = pd.read_csv(col_dir)

drop_cols = ['overduedays','submittedtime','customrepaymentamount', 'cutoff','jiraarchived','telecredittime','thirdpartyid','selfiekey','jirakey','selfiechecktime','repaidamount', 'repaymenttime','premiumprocessfee','fraudmetrixrequest','fraudmetrix', 'refid','verificationtime','punishfee','notifylevel','userconfirmtime','rate','totalfee', 'debitcardid', 'operateservicefee', 'rejectdetail', 'laststatuschangedtime', 'auditstaff', 'audittype', 'repaymentway', 'withdrawcardid', 'couponid', 'repaymentchannel', 'originalloanid', 'passtoauditorstafftime', 'loantype', 'discountfee', 'created', 'deleted', 'updated', 'version', 'canceltime', 'decisiontime', 'rejectreason']

remain_cols = set(one_month.columns) - set(drop_cols)

df = one_month[remain_cols]

df[df.payofftime.isnull()]

In [10]:
one_month.head()

,id,created,deleted,updated,version,amount,canceltime,days,decisiontime,fundtime,...,rejectreason,fraudmetrixrequest,originalloanid,customrepaymentamount,repaidamount,payoffclearedtime,verificationtime,selfiechecktime,jiraarchived,thirdpartyid
0,75831996,2018-06-08 16:39:07,NaN,2018-06-08 16:43:31,9,1000,NaN,150,2018-06-08 16:39:54,2018-06-08 16:43:31,...,NaN,NaN,75831996,0,0,NaN,2018-06-08 16:40:52,NaN,NaN,NaN
1,75832034,2018-06-08 16:40:10,NaN,2018-06-08 16:46:51,9,2000,NaN,150,2018-06-08 16:43:43,2018-06-08 16:46:51,...,NaN,NaN,75832034,0,0,NaN,2018-06-08 16:44:09,NaN,NaN,NaN
2,75832063,2018-06-08 16:40:46,NaN,2018-07-08 00:42:44,10,2000,NaN,150,2018-06-08 16:50:05,2018-06-08 16:53:25,...,NaN,NaN,75832063,0,0,2018-07-08 00:42:43,2018-06-08 16:51:36,NaN,NaN,NaN
3,75832085,2018-06-08 16:41:19,NaN,2018-06-08 16:44:58,9,1000,NaN,150,2018-06-08 16:41:56,2018-06-08 16:44:58,...,NaN,NaN,75832085,0,0,NaN,2018-06-08 16:43:20,NaN,NaN,NaN
4,75832098,2018-06-08 16:41:37,NaN,2018-11-04 18:51:07,10,1000,NaN,150,2018-06-08 16:44:55,2018-06-08 16:49:22,...,NaN,NaN,75832098,0,0,2018-11-04 18:51:07,2018-06-08 16:47:35,NaN,NaN,NaN


In [2]:
import lightgbm as lgb
import json
import scipy
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
# from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv,GCNConv


import torch
from torch_geometric.data import InMemoryDataset, download_url
# from torch_geometric.read import read_planetoid_data
def sample_mask(index, num_nodes):
    mask = torch.zeros((num_nodes, ), dtype=torch.uint8)
    mask[index] = 1
    return mask

def split_idx(y, split):
    if isinstance(y, torch.Tensor):
        y = torch.tensor(y)
    x_perm = y[torch.randperm(len(y))]
    train_end_idx = int(len(y)*split[0])
    valid_end_idx = int(len(y)*(split[0]+split[1]))
    train_idx = x_perm[:train_end_idx]
    valid_idx = x_perm[train_end_idx:valid_end_idx]
    test_idx = x_perm[valid_end_idx:]
    return train_idx, valid_idx, test_idx

def samples_split_mask(y, split=(0.6,0.2)):
    train_idx, valid_idx, test_idx = split_idx(y, split)
    train_mask = sample_mask(train_idx, num_nodes=y.size(0))
    val_mask = sample_mask(valid_idx, num_nodes=y.size(0))
    test_mask = sample_mask(test_idx, num_nodes=y.size(0))
    return train_mask, val_mask, test_mask

def read_cashbus_data(root):
    # -- userid2idx;
    with open(root + '/col_4_dict', 'r') as f:
        userid2idx = json.load(f)
    # -- node_fetas;  labels; edges;
    x = sparse.load_npz(root + "/feat.npz")
    y = sparse.load_npz(root + "/y.npz")
    edges = sparse.load_npz(root + "/edges.npz").toarray().T
    
    x = torch.tensor(x.toarray(), dtype =torch.float)
    y = torch.tensor(y.toarray()).squeeze()
    row1 = [userid2idx[str(i)] for i in edges[0]]
    row2 = [userid2idx[str(i)] for i in edges[1]]
    new_edges = torch.tensor(np.stack([row1, row2]))
    train_mask, val_mask, test_mask = samples_split_mask(y, split=(0.6,0.2))
    data = Data(x=x, edge_index=new_edges, y=y)
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask
    return data


class CashBus(InMemoryDataset):
    r"""The citation network datasets "Cora", "CiteSeer" and "PubMed" from the
    `"Revisiting Semi-Supervised Learning with Graph Embeddings"
    <https://arxiv.org/abs/1603.08861>`_ paper.
    Nodes represent documents and edges represent citation links.
    Training, validation and test splits are given by binary masks.
    Args:
        root (string): Root directory where the dataset should be saved.
        name (string): The name of the dataset (:obj:`"Cora"`,
            :obj:`"CiteSeer"`, :obj:`"PubMed"`).
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
    """


    def __init__(self, root, transform=None, pre_transform=None):
        super(CashBus, self).__init__(root, transform, pre_transform)
        print('processed_path:{}'.format(self.processed_paths))
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['col_4_dict', 'feat.npz', 'y.npz', 'edges.npz', 'col_name.pkl']

    @property
    def processed_file_names(self):
        return 'data.pt'
    
    def download(self):
        pass

    def process(self):
        print('go pl, raw_dir:{}'.format(self.raw_dir))
        data = read_cashbus_data(self.raw_dir)
        data = data if self.pre_transform is None else self.pre_transform(data)
        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

    def __repr__(self):
        return '{}()'.format(self.name)

1. only use node_feats and lgb, construct the same test set;

In [5]:
root = '/data-0/qibo/Gdata/col_4/raw'

with open(root + '/col_4_dict', 'r') as f:
    userid2idx = json.load(f)
with open(root + '/col_name.pkl', 'rb') as f:
    col_names = pickle.load(f)
# -- node_fetas;  labels; edges;
x = sparse.load_npz(root + "/feat.npz")
y = sparse.load_npz(root + "/y.npz")
edges = sparse.load_npz(root + "/edges.npz").toarray().T
df = pd.DataFrame(x.toarray(), columns=col_names)

2. 只用点特征 入lgb;

作为反欺诈，是解决冷启动的方式之一，可是我训练的都是老用户啊；

引出欺诈对象，欺诈样本的设定 --> 打一枪就跑；和 打N枪就跑；

In [28]:
oneshot = df[(df.paidoff==0)&(df.fund==1)]
fewshot = df[(df.paidoff!=df.fund)&(df.fund!=0)]
zeroshot = df[df.fund==0]

In [ ]:
def feat_lables(oneshot):
    labels = (train_lgb.fund == train_lgb.paidoff)*1
    feats = train_lgb.drop('paidoff', axis=1).values

In [9]:
train_lgb = df[df.fund!=0]
labels = (train_lgb.fund == train_lgb.paidoff)*1
feats = train_lgb.drop('paidoff', axis=1).values

In [22]:
def lgb_cv(Final_x,Final_y):
    cv_train = lgb.Dataset(Final_x, Final_y.astype('int'))
    params = {'task': 'train',
              'boosting_type': 'gbdt',
              'objective': 'binary',
              'metric': 'auc',
              'num_threads': 10,
              'num_leaves': 3,  # 31,
              'learning_rate': 0.01,  # 0.002
              'feature_fraction': 0.5,
              'lambda_l2': 140,
              'bagging_fraction': 0.5,
              'bagging_freq': 5}
    gbm = lgb.cv(params,
                 cv_train,
                 num_boost_round=5000,
                 nfold=5,
                 early_stopping_rounds=100,
                 verbose_eval=200)
    return gbm['auc-mean'][-1], gbm['auc-stdv'][-1]

In [23]:
lgb_cv(feats, labels)

[200]	cv_agg's auc: 0.759032 + 0.00376798
[400]	cv_agg's auc: 0.760467 + 0.00342344
[600]	cv_agg's auc: 0.761317 + 0.0033297
[800]	cv_agg's auc: 0.762391 + 0.0032067
[1000]	cv_agg's auc: 0.762801 + 0.00318433
[1200]	cv_agg's auc: 0.763286 + 0.0033135
[1400]	cv_agg's auc: 0.763712 + 0.00349232
[1600]	cv_agg's auc: 0.76424 + 0.00369247
[1800]	cv_agg's auc: 0.765003 + 0.00362209
[2000]	cv_agg's auc: 0.765442 + 0.00371491
[2200]	cv_agg's auc: 0.766006 + 0.00360971
[2400]	cv_agg's auc: 0.766568 + 0.0033563
[2600]	cv_agg's auc: 0.767051 + 0.00341389
[2800]	cv_agg's auc: 0.767407 + 0.00333975
[3000]	cv_agg's auc: 0.767567 + 0.00348625
[3200]	cv_agg's auc: 0.768032 + 0.00343956
[3400]	cv_agg's auc: 0.76826 + 0.00342501
[3600]	cv_agg's auc: 0.768551 + 0.00348826
[3800]	cv_agg's auc: 0.768733 + 0.00346285
[4000]	cv_agg's auc: 0.768881 + 0.00343265
[4200]	cv_agg's auc: 0.769028 + 0.00344021
[4400]	cv_agg's auc: 0.769198 + 0.00341949
[4600]	cv_agg's auc: 0.76936 + 0.00344211
[4800]	cv_agg's auc: 0

(0.7695653570803928, 0.0034532034930151554)

In [ ]:
信息泄露 + 用户分层，有一些层级用户直接就可以推断出来；

In [ ]:
def read_cashbus_data(root):
    # -- userid2idx;
    with open(root + '/col_4_dict', 'r') as f:
        userid2idx = json.load(f)
    # -- node_fetas;  labels; edges;
    x = sparse.load_npz(root + "/feat.npz")
    y = sparse.load_npz(root + "/y.npz")
    edges = sparse.load_npz(root + "/edges.npz").toarray().T
    return x, y, edges

In [ ]:
class CashBus(InMemoryDataset):
    r"""The citation network datasets "Cora", "CiteSeer" and "PubMed" from the
    `"Revisiting Semi-Supervised Learning with Graph Embeddings"
    <https://arxiv.org/abs/1603.08861>`_ paper.
    Nodes represent documents and edges represent citation links.
    Training, validation and test splits are given by binary masks.
    Args:
        root (string): Root directory where the dataset should be saved.
        name (string): The name of the dataset (:obj:`"Cora"`,
            :obj:`"CiteSeer"`, :obj:`"PubMed"`).
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
    """


    def __init__(self, root, transform=None, pre_transform=None):
        super(CashBus, self).__init__(root, transform, pre_transform)
        print('processed_path:{}'.format(self.processed_paths))
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['col_4_dict', 'feat.npz', 'y.npz', 'edges.npz', 'col_name.pkl']

    @property
    def processed_file_names(self):
        return 'data.pt'
    
    def download(self):
        pass

    def process(self):
        print('go pl, raw_dir:{}'.format(self.raw_dir))
        data = read_cashbus_data(self.raw_dir)
        data = data if self.pre_transform is None else self.pre_transform(data)
        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

    def __repr__(self):
        return '{}()'.format(self.name)

In [3]:
def lgb_cv(Final_x,Final_y):
    cv_train = lgb.Dataset(Final_x, Final_y.astype('int'))
    params = {'task': 'train',
              'boosting_type': 'gbdt',
              'objective': 'binary',
              'metric': 'auc',
              'num_threads': 10,
              'num_leaves': 3,  # 31,
              'learning_rate': 0.008,  # 0.002
              'feature_fraction': 0.5,
              'lambda_l2': 140,
              'bagging_fraction': 0.5,
              'bagging_freq': 5}
    gbm = lgb.cv(params,
                 cv_train,
                 num_boost_round=5000,
                 nfold=5,
                 early_stopping_rounds=100,
                 verbose_eval=200)
    return gbm['auc-mean'][-1], gbm['auc-stdv'][-1]